<a href="https://colab.research.google.com/github/0xfeedbac/KODE/blob/master/e3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST and Beyond

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/r2/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

The 'try' block is needed for colab to use 2.0 instead of 1.4



In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

Work with CIFAR instead of original MNIST

In [0]:
cifar = tf.keras.datasets.cifar10
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets

(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Define and train a very basic sequantial model.

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
# comment about input shape
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs=3)

model.evaluate(x_test, y_test, verbose=0)

The accuracy is only .31, which means a lot of confusion. 

Obtain confusion matrix:

In [0]:
# one of the Sequential model's methods return predictions directly in a form of classes https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
y_predicted = model.predict_classes(x_test)

# flatten tensor using tf instruments https://stackoverflow.com/questions/34194151/best-way-to-flatten-a-2d-tensor-containing-a-vector-in-tensorflow
y_true = tf.squeeze(y_test)

cm = tf.math.confusion_matrix(y_true, y_predicted)

print(cm)

Represent confusion matrix (cm) in a more human-oriented way.

In [0]:
from sklearn.preprocessing import normalize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Transform tf tensor to numpy array for plotting 
# Normalize to make fractions out of absolute occurance counts
cm_normalized = normalize(cm.numpy(), axis=1, norm='l1') 
# The default normalization is l2, which inflates low values, so that 
# percents do not add up to 100.
# http://mathworld.wolfram.com/L2-Norm.html
# http://mathworld.wolfram.com/L1-Norm.html

classes = ['Airplane','Car','Bird','Cat','Deer',
           'Dog','Frog','Horse','Ship','Truck']

df_cm = pd.DataFrame(cm_normalized, columns=classes, index=classes)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (10,9))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, annot=True, fmt='.0%')

The cat is a combination of Dog, Frog, Bird and Car. 

Some ideas about how-to improve the model by adding convolutional layers: https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5

The TF2.0 API for convolution layers is located here: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D  
Example of how-to use them: https://www.tensorflow.org/beta/tutorials/quickstart/advanced
